## データの読み込み

In [2]:
import pandas as pd
import numpy as np

#local 読み込み
dataPath = "datasets/"
keyPath = 'data/keyworddata/'

## keywordを抽出

In [2]:
test = pd.read_csv(dataPath + 'test.csv')
train = pd.read_csv(dataPath +'train.csv')
sub = pd.read_csv(dataPath +'sample_submission.csv')

data = pd.concat([train, test], sort=False)#データの統合

data['keyword'].to_csv(keyPath + 'origin.csv')

In [9]:
from mojimoji import zen_to_han
import re

#unicode正規化
import unicodedata
def normalize_unicode(text):
  return unicodedata.normalize('NFKC', text)

#記号,特殊文字を削除(数字, 英語, ひらがな,カタカナ,漢字,ー。、以外'、'に変換)
def delete_symbol(text):
  return re.sub('[^0-9a-zA-Zぁ-んァ-ヿｦ-ﾟ一-龥ー。、]', '',text)

#データのクリーン処理
def clean_text(text):
  wL = []
  text = str(text)
  for w in text.split():
    w = normalize_unicode(w)
    w = delete_symbol(w)
    wL.append(w)
  return " ".join(wL)

#titleをクリーニング処理
data = pd.read_csv(keyPath+'origin.csv', index_col=0)
data['cleaning'] = data['keyword'].apply(clean_text)
data.to_csv(keyPath+'cleaning.csv')

In [9]:
from sudachipy import tokenizer
from sudachipy import dictionary

tokenizer_obj = dictionary.Dictionary().create()

# sudachi による単語への分割関数,(名詞,形容詞,動詞)のみ残す
def AnalysisbySudachi(text):
  if text is np.nan:#欠損値は欠損値のまま返す
    return np.NaN
  mode = tokenizer.Tokenizer.SplitMode.C #モードCの一番長い形で分ける
  results =[m.surface() for m in tokenizer_obj.tokenize(text, mode)]
  word_list = []
  for word in results:
    if not (word == ""): #何故か分かち書きの結果として空白データ（''）ができたための省く処理
      normalize = tokenizer_obj.tokenize(word, mode)[0].normalized_form() #正規化（標準化？）してなるべく言葉の揺れを無くす　e.g. 打込む → 打ち込む かつ丼 → カツ丼
      pos = tokenizer_obj.tokenize(normalize, mode)[0].part_of_speech()[0]
      if pos in  ["名詞", "動詞", "形容詞"]:  # 対象とする品詞を指定
        word = tokenizer_obj.tokenize(normalize, mode)[0].dictionary_form()
        word_list.append(word)
  return " ".join(word_list) #スペースで繋げていく

#形態素解析結果を'sudachi'に代入
data = pd.read_csv(keyPath+'cleaning.csv', index_col=0)
data['sudachi'] = data['cleaning'].apply(AnalysisbySudachi)
data.to_csv(keyPath+'sudachi.csv')


トゥデー 晩御飯
トゥデー 晩御飯 女子高生
トゥデー 晩御飯 カツ丼


## 分散表現に変換

### 学習済みモデルの読み込み

In [2]:
import gensim
from pprint import pprint

# chiVeデータのPATH（kv:KeyedVectors）
model_path = './jalang/entity_vector.model.txt'
# モデルの読み込み
model = gensim.models.KeyedVectors.load_word2vec_format(model_path)

### 自作学習モデル

In [ ]:
from gensim.models import word2vec

data = pd.read_csv(keyPath+'origin.csv', index_col=0)

#[[文章の単語], [文章の単語]]を作成
title_dic = []
def make_title_dic(text):
  if text is np.nan:
    return
  title_dic.append(text.split(' '))
data['keyword'].map(make_title_dic)

#学習
model = word2vec.Word2Vec(title_dic,vector_size=200, min_count=5, window=5, epochs=20)

#作成した辞書をcsvに保存して可視化(処理には関係ない)
from gensim import corpora#辞書を作るためのもの
dictionary = corpora.Dictionary(title_dic)#textsをもとに辞書を作成します
dictionary.filter_extremes(no_below=5)#出現文書数が5回以下のものはさようなら
dictionary.save_as_text(keyPath + 'dictionary.csv')#辞書を保存
model = model.wv
# In[4]:
print('語彙の数:', len(model.index_to_key))  # 語彙の数
print(model.index_to_key[:10])

### 分散表現に変換

In [ ]:
#文章の分散表現を求める(単語ベクトルの平均)
def getSentenceVector(text):
  if text is np.nan:
    return np.array([np.nan for _ in range(200)])
  L = []
  nonL = []
  for w in text.split(' '):
    if w in model.key_to_index:
      L.append(model.get_vector(key=w))
    else:
      nonL.append(w)#辞書に場合
  #辞書にないものを出力
  with open('./data/keyworddata/nonDic.txt', 'w') as f:
    for d in nonL:
      try:
        f.write("%s\n" % d)
      except Exception as e:
        print(d, e)
  if len(L)==0:
    return np.array([np.nan for _ in range(200)])
  return np.array(L).mean(axis=0)

#分散表現をnp.arrayの形に変換
def makeVectorArrayList(data):
  vec_list = []
  for index, row in data.iterrows():
    vec_list.append(getSentenceVector(row['keyword']))
  return np.array(vec_list)

data = pd.read_csv(keyPath+'origin.csv', index_col=0)
vec_list_array = makeVectorArrayList(data)
vec_data = pd.DataFrame(data=vec_list_array, dtype='float')
vec_data.to_csv(keyPath+'key_vec_originalModel.csv')


In [7]:
len(vec_data)

48522